In [80]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.preprocessing import OneHotEncoder

### Загрузка и подготовка данных

In [81]:
data = pd.read_csv('data_ford_price.csv',sep=';')
data=data[['price','year','condition','cylinders','odometer','lat','long','weather']]
data['weather'].fillna(np.round(np.mean(data['weather']),0))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   price      7017 non-null   int64  
 1   year       7017 non-null   int64  
 2   condition  7017 non-null   int64  
 3   cylinders  7017 non-null   int64  
 4   odometer   7017 non-null   int64  
 5   lat        7017 non-null   float64
 6   long       7017 non-null   float64
 7   weather    6837 non-null   float64
dtypes: float64(3), int64(5)
memory usage: 438.7 KB


In [82]:
y = data['price']
x = data.drop(columns='price')

x = x.dropna()
y = y.iloc[x.index]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Метод рекурсивного исключения признаков (RFE)

In [83]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()
categorial_columns_rfe = selector.get_feature_names_out().tolist()
print(categorial_columns_rfe)

['year', 'condition', 'cylinders']


In [84]:
model = LinearRegression()
model.fit(X_train[categorial_columns_rfe], y_train)
y_predicted = model.predict(X_test[categorial_columns_rfe])
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE_rfe: %.3f' % mae)

MAE_rfe: 5176.148


### ВЫБОР ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ (выбор k лучших переменных: SelectKBest)

In [85]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
print(selector.get_feature_names_out())

['year' 'condition' 'odometer']


In [86]:
categorial_columns_skb = selector.get_feature_names_out().tolist()
model = LinearRegression()
model.fit(X_train[categorial_columns_skb], y_train)
y_predicted = model.predict(X_test[categorial_columns_skb])
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE_skb: %.3f' % mae)

MAE_skb: 4913.526


#### По результатам подсчета МАЕ метод отбора признаков на основе фильтров показал лучший результат, чем RFE